In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=d9a6db9e28fc102a5e6c4b571567614925d1e3ed2994adddcf23c2e988ae40d8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("case_study_sql").getOrCreate()

Create DataFrame

In [5]:
data = [('Alice', 27, 25000),('Tom', 25, 30000),('Jack', 22, 19000),('Daniel', 30, 35000)]

In [6]:
sqlDF = spark.createDataFrame(data, ['name', 'age', 'salary'])

In [7]:
sqlDF.show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Alice| 27| 25000|
|   Tom| 25| 30000|
|  Jack| 22| 19000|
|Daniel| 30| 35000|
+------+---+------+



In [8]:
sqlDF.createOrReplaceTempView("person")

In [9]:
spark.sql("SELECT * from person").show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Alice| 27| 25000|
|   Tom| 25| 30000|
|  Jack| 22| 19000|
|Daniel| 30| 35000|
+------+---+------+



In [10]:
newDF = spark.table("person")

In [11]:
newDF.show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Alice| 27| 25000|
|   Tom| 25| 30000|
|  Jack| 22| 19000|
|Daniel| 30| 35000|
+------+---+------+



In [12]:
sqlDF.registerTempTable("person1")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [13]:
spark.sql("select * from person1").show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Alice| 27| 25000|
|   Tom| 25| 30000|
|  Jack| 22| 19000|
|Daniel| 30| 35000|
+------+---+------+



In [14]:
spark.catalog.dropTempView("person")

True

Some SQL queries

In [17]:
spark.sql("SELECT name,age from person1").show()

+------+---+
|  name|age|
+------+---+
| Alice| 27|
|   Tom| 25|
|  Jack| 22|
|Daniel| 30|
+------+---+



In [18]:
spark.sql("SELECT max(salary) from person1").show()

+-----------+
|max(salary)|
+-----------+
|      35000|
+-----------+



In [19]:
spark.sql("SELECT avg(salary) from person1").show()

+-----------+
|avg(salary)|
+-----------+
|    27250.0|
+-----------+



In [20]:
spark.sql("SELECT sum(salary) as Total_Salary from person1").show()

+------------+
|Total_Salary|
+------------+
|      109000|
+------------+



In [21]:
spark.sql("SELECT * from person1 where salary > 20000").show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Alice| 27| 25000|
|   Tom| 25| 30000|
|Daniel| 30| 35000|
+------+---+------+



In [22]:
spark.sql("SELECT * from person1 where salary > 20000 and age > 25").show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Alice| 27| 25000|
|Daniel| 30| 35000|
+------+---+------+



JOINS

In [23]:
x = [('Pirate',1),('Monkey',2),('Ninja',7),('Spaghetti',4)]
df1 = spark.createDataFrame(x,['name','id'])

y = [('Rutabaga',1),('Pirate',5),('Ninja',3),('Darth Vader',4)]
df2 = spark.createDataFrame(y,['name','id'])


In [24]:
df1.registerTempTable("table1")
df2.registerTempTable("table2")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [25]:
spark.sql("SELECT table1.*,table2.* FROM table2 inner JOIN table1 ON table2.id = table1.id").show()

+---------+---+-----------+---+
|     name| id|       name| id|
+---------+---+-----------+---+
|   Pirate|  1|   Rutabaga|  1|
|Spaghetti|  4|Darth Vader|  4|
+---------+---+-----------+---+



In [26]:
spark.sql("SELECT table1.*,table2.* FROM table2 left JOIN table1 ON table2.id = table1.id").show()

+---------+----+-----------+---+
|     name|  id|       name| id|
+---------+----+-----------+---+
|     NULL|NULL|     Pirate|  5|
|   Pirate|   1|   Rutabaga|  1|
|     NULL|NULL|      Ninja|  3|
|Spaghetti|   4|Darth Vader|  4|
+---------+----+-----------+---+



In [27]:
spark.sql("SELECT table1.*,table2.* FROM table2 right JOIN table1 ON table2.id = table1.id").show()

+---------+---+-----------+----+
|     name| id|       name|  id|
+---------+---+-----------+----+
|   Pirate|  1|   Rutabaga|   1|
|   Monkey|  2|       NULL|NULL|
|    Ninja|  7|       NULL|NULL|
|Spaghetti|  4|Darth Vader|   4|
+---------+---+-----------+----+



In [28]:
spark.sql("SELECT table1.*,table2.* FROM table2 full JOIN table1 ON table2.id = table1.id").show()

+---------+----+-----------+----+
|     name|  id|       name|  id|
+---------+----+-----------+----+
|   Pirate|   1|   Rutabaga|   1|
|   Monkey|   2|       NULL|NULL|
|     NULL|NULL|      Ninja|   3|
|Spaghetti|   4|Darth Vader|   4|
|     NULL|NULL|     Pirate|   5|
|    Ninja|   7|       NULL|NULL|
+---------+----+-----------+----+



In [29]:
spark.sql("SELECT table2.* FROM table2 left anti JOIN table1 ON table2.id = table1.id").show()

+------+---+
|  name| id|
+------+---+
|Pirate|  5|
| Ninja|  3|
+------+---+



In [30]:
spark.sql("SELECT table2.* FROM table2 left semi JOIN table1 ON table2.id = table1.id").show()

+-----------+---+
|       name| id|
+-----------+---+
|   Rutabaga|  1|
|Darth Vader|  4|
+-----------+---+

